In [3]:
import os
import getpass
from dotenv import load_dotenv
from pydantic import BaseModel, Field, validator
load_dotenv()
from langchain.output_parsers import PydanticOutputParser
import os, time
from langchain_openai.chat_models import ChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from typing import TypedDict, Annotated, List, Union
from langchain_core.agents import AgentAction, AgentFinish
from langchain_core.messages import BaseMessage
import operator
from langchain_core.agents import AgentFinish
from langgraph.prebuilt.tool_executor import ToolExecutor
from langgraph.graph import END, StateGraph
import json

import functools
from langchain_core.messages import (
    AIMessage,
    BaseMessage,
    ChatMessage,
    FunctionMessage,
    HumanMessage,
)
from langchain.tools.render import format_tool_to_openai_function
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langgraph.graph import END, StateGraph
import operator
from typing import Annotated, List, Sequence, Tuple, TypedDict, Union
from langgraph.prebuilt.tool_executor import ToolExecutor, ToolInvocation


In [4]:
#define graph state
class AgentState(TypedDict):
    chat_history: list[BaseMessage]
    messages: Annotated[Sequence[BaseMessage], operator.add]
    # sender: str
    user_config:dict


class OutputObject(BaseModel):
    response: str = Field(description="your contribution or response; this should be empty when you are raising your hand(s)")
    hand: bool = Field(description="set to true if your hand is raised else false", default=False)
    sender: str = Field(description="your name")
    directed_to: str = Field(description="if your response is directed to anyone specific in the room, your specify that here using their name else set this to general", default="general")
    go_ahead: bool = Field(description="set to true if you want the expert to speak (This is for the moderator only)", default=False)
    
    def to_json(self):
        return json.dumps(self.dict())

# Set up a parser 
pydantic_parser = PydanticOutputParser(pydantic_object=OutputObject)
format_instructions = pydantic_parser.get_format_instructions()




class OutputObject2(BaseModel):
    response: str = Field(description="your response/message")
    sender: str = Field(description="your name")
    directed_to: str = Field(description="if your response is directed to anyone specific in the room, your specify that here using their name else set this to general", default="general")
    go_ahead: bool = Field(description="set to true if you want the expert to speak", default=False)
    
    def to_json(self):
        return json.dumps(self.dict())

# Set up a parser 
pydantic_parser = PydanticOutputParser(pydantic_object=OutputObject2)
moderator_format_instructions = pydantic_parser.get_format_instructions()

other_experts = [{
    "name":"mr remi",
    "field": "solar and inverter"},
                 
    {
    "name":"mr bayo",
    "field": "smart lights"},
    
    {"name":"moderator",
    "field": "moderating the conversation"},
                 
                 ]




def create_agent(llm, avatar,expert_in, tools=[], experts=other_experts, response_format=format_instructions, system_message: str=None):
    """Create an agent."""
    functions = [format_tool_to_openai_function(t) for t in tools]

    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You name is {avatar} (an expert in  {expert_in}), in a room with other experts on diffrent fields." 
                "You are gathered in this room to just share knowlege and talk about any topic that might arise."
                "There is a moderator that oversees the conversaton, so when you feel you have something to contribute,"
                "raise up your hands only (do not speak yet),"
                "then you wait for the moderator to give you a 'go_ahead' to speak before you do."
                "Specify if the response is to everyone or directed to a specific expert,"
                "Here is the format for every single message you send: {response_format}."
                "Here is the complete list of experts in the room {experts}"
                
                
                #" If you or any of the other assistants have the final answer or deliverable i.e the test cases,"
                #" prefix your response with FINAL ANSWER so the team knows to stop."
                #" You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(avatar=avatar)
    prompt = prompt.partial(expert_in=expert_in)
    prompt = prompt.partial(response_format=response_format)
    prompt = prompt.partial(experts=experts)
    #prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm #.bind_functions(functions)






# Helper function to create a node for a given agent
def agent_node(state, agent, name):
    result = agent.invoke(state)
    # We convert the agent output into a format that is suitable to append to the global state
    if isinstance(result, FunctionMessage):
        pass
    else:
        result = HumanMessage(**result.dict(exclude={"type", "name"}), name=name)
    return {
        "messages": [result],
        # Since we have a strict workflow, we can
        # track the sender so we know who to pass to next.
        #"sender": name,
    }

In [3]:
#SOLAR SYSTEM AND INVERTER EXPERT
llm = ChatOpenAI(model="gpt-4-1106-preview", model_kwargs = {"response_format":{"type": "json_object"}})
#system_message_=""
solar_expert = create_agent(
    llm=llm,
    avatar="mr remi",
    expert_in = "solar/inverter solutions for homes"
)
solar_expert_node = functools.partial(agent_node, agent=solar_expert, name="mr remi")



#SMART LIGHTS EXPERT
llm = ChatOpenAI(model="gpt-4-1106-preview", model_kwargs = {"response_format":{"type": "json_object"}})
#system_message_=""
lights_expert = create_agent(
    llm=llm,
    avatar="mr bayo",
    expert_in = "smart lights expert"
)
lights_expert_node = functools.partial(agent_node, agent=lights_expert, name="mr bayo")



#MODERATOR
llm_moderator = ChatOpenAI(model="gpt-4-1106-preview", model_kwargs = {"response_format":{"type": "json_object"}})
"""Create an agent."""
def mod(llm=llm_moderator, avatar="moderator",expert_in="organising conversations", tools=[], experts=other_experts, response_format=moderator_format_instructions, system_message: str=None):
    prompt = ChatPromptTemplate.from_messages(
        [
            (
                "system",
                "You name is {avatar} (an expert in  {expert_in}), in a room with other experts on diffrent fields. " 
                "You are gathered in this room to just share knowlege and talk about any topic that might arise. "
                "You are the moderator that oversees the conversaton. "
                "You are to decide what to do next, or who to speak next. "
                "Experts will raise up their hands only, when they want to say something, "
                "it is your job to give a go_ahead if you want them to speak next. "
                "you must Specify who yourresponse is; any of the experts. "
                
                "Here is the format for every single message you send: {response_format}. "
                "Here is the complete list of experts in the room {experts}. "
                "For now, just let them take turns sequentially as the appear, and start all over again."
                
                
                #" If you or any of the other assistants have the final answer or deliverable i.e the test cases,"
                #" prefix your response with FINAL ANSWER so the team knows to stop."
                #" You have access to the following tools: {tool_names}.\n{system_message}",
            ),
            MessagesPlaceholder(variable_name="messages"),
        ]
    )
    prompt = prompt.partial(avatar=avatar)
    prompt = prompt.partial(expert_in=expert_in)
    prompt = prompt.partial(response_format=response_format)
    prompt = prompt.partial(experts=experts)
    #prompt = prompt.partial(tool_names=", ".join([tool.name for tool in tools]))
    return prompt | llm #.bind_functions(functions)


moderator = mod()
moderator_node = functools.partial(agent_node, agent=moderator, name="moderator")


In [6]:
import random
experts_names = [expert["name"] for expert in other_experts]
experts_names.remove("moderator")

#Edges logic
def moderator_to_expert_edge_logic(state):
    # This is the router
    messages = state["messages"]
    last_message = json.loads(messages[-1].content)
    
    
    if last_message["sender"] !="moderator": #any of the experts
        return "moderator" #send to moderator for broadcast
    elif last_message['directed_to'] =="general" : #moderator directs message to general then ramdomly pick an expert
         return random.choice(experts_names)
    else:
        return last_message['directed_to'] #send to who it is meant for
      

In [5]:
workflow = StateGraph(AgentState)
workflow.add_node("mr bayo",lights_expert_node)
workflow.add_node("moderator",moderator_node)
workflow.add_node("mr remi",solar_expert_node)
workflow.set_entry_point("moderator")


#EDGES
workflow.add_conditional_edges(
    "moderator",
    moderator_to_expert_edge_logic,
    {expert: expert for expert in experts_names} #go to who the message was directed to
)



for i, expert in enumerate(experts_names):
    workflow.add_edge(
    expert,
    "moderator"
)

graph = workflow.compile()

In [24]:
start_message = {
  "topic": "The topic is 'Does having a smart home worth it?' ",
}

for s in graph.stream(
    {
        "messages": [
            HumanMessage(
                content=f'''{start_message["topic"]}'''
            )
        ],
        
    },
    # Maximum number of steps to take in the graph
    {"recursion_limit": 100},
):
    name = list(s.keys())[0]
    content = json.loads(s[name]["messages"][0].content)["response"]
    try:
        hand = json.loads(s[name]["messages"][0].content)["hand"]
    except:
        hand = False
    print("Name:", name)
    if hand:
        print("Notification:", f''' {name} is raising his/her hands''')
    if content !="":
          print("Response:", content)
    
    print("----")

Name: moderator
Response: Thank you for the topic. Let's explore the value of having a smart home. Mr. Remi, with your expertise in solar and inverter technology, can you share your insights on how smart homes might integrate renewable energy solutions? Your perspective would be valuable to this discussion.
----


In [7]:
ee = {expert: expert for expert in experts_names}
ee.update({"end":END})

In [8]:
ee

{'mr remi': 'mr remi', 'mr bayo': 'mr bayo', 'end': '__end__'}